In [1]:
%reset -f

%load_ext autoreload
%autoreload 2

import pandas as pd

from functions import *

datasets = {}
for hour in range(24):
    df = pd.read_csv(f"Splits/dataset_hour_{hour}.csv")

    datasets[hour] = deseasonalize(df, "both")

In [2]:
data_baseline = pd.read_csv('combined_predictions.csv')
data_ml = pd.read_csv('combined_predictions_ml.csv')
data_darma = pd.read_csv('combined_predictions_darma.csv')

models_baseline = np.concatenate(
    [[out_of_sample_pred(
        np.array(data_baseline["Actual"]).reshape(-1, 1), 
        np.array(data_baseline[f"Model_{i}"]).reshape(-1, 1)
    )] for i in range(1, 7)],
)

model_ml = [out_of_sample_pred(np.array(data_baseline["Actual"]).reshape(-1, 1),
                               np.array(data_ml[f"Predictions"]).reshape(-1, 1))]

models_darma = np.concatenate(
    [[out_of_sample_pred(
        np.array(data_baseline["Actual"]).reshape(-1, 1), 
        np.array(data_darma[f"Model_{i}"]).reshape(-1, 1)
    )] for i in range(1, 4)],
)


In [4]:
all_models = np.concatenate([models_baseline, model_ml, models_darma])

model_names = [
    'AR(1) with constant and first differences',
    'AR(7) with constant and first differences',
    'ARIMA(1,1,1) with constant',
    'ARIMAX(1,1,1) with constant',
    'ARIMAX(7,1,1) with constant',
    'Seasonal ARIMAX(1,1,1,7) with constant (non-seasonal data)',
    'Lasso Estimated Autoregression',
    'DAR(3)',
    'DARMA(1,1)',
    'DARMAX(1,1)'
]

df = pd.DataFrame(all_models, columns=['RMSE', 'SMAPE', 'RMAE'])
df['Model'] = model_names
df = df[['Model', 'RMSE', 'SMAPE', 'RMAE']]

# Identify min values
min_rmse = df['RMSE'].min()
min_smape = df['SMAPE'].min()
min_rmae = df['RMAE'].min()

# Bold formatting for min values
df['RMSE'] = df['RMSE'].apply(lambda x: '\\textbf{{{:.1f}}}'.format(x) if x == min_rmse else '{:.1f}'.format(x))
df['SMAPE'] = df['SMAPE'].apply(lambda x: '\\textbf{{{:.2f}}}'.format(x) if x == min_smape else '{:.2f}'.format(x))
df['RMAE'] = df['RMAE'].apply(lambda x: '\\textbf{{{:.3f}}}'.format(x) if x == min_rmae else '{:.3f}'.format(x))

# Print LaTeX
print(df.to_latex(index=False, escape=False))

\begin{tabular}{llll}
\toprule
Model & RMSE & SMAPE & RMAE \\
\midrule
AR(1) with constant and first differences & 128.5 & 47.73 & 75.636 \\
AR(7) with constant and first differences & 119.1 & 44.34 & 72.809 \\
ARIMA(1,1,1) with constant & 116.8 & 46.14 & 72.602 \\
ARIMAX(1,1,1) with constant & 116.2 & 46.10 & 72.224 \\
ARIMAX(7,1,1) with constant & 118.6 & 44.24 & 73.192 \\
Seasonal ARIMAX(1,1,1,7) with constant (non-seasonal data) & \textbf{112.4} & \textbf{41.19} & \textbf{69.195} \\
Lasso Estimated Autoregression & 130.2 & 47.57 & 81.471 \\
DAR(3) & 125.0 & 46.33 & 74.303 \\
DARMA(1,1) & 130.3 & 45.70 & 75.249 \\
DARMAX(1,1) & 130.0 & 45.63 & 75.191 \\
\bottomrule
\end{tabular}



In [14]:
dataset_date = [datasets[hour]['DATE'] for hour in range(24)]

datasets_dates = []

n_days = min(len(datasets[hour]) for hour in range(24))

combined_predictions = []

for day in range(n_days):
    for hour in range(24):
        datasets_dates.append(dataset_date[hour][day])

time = np.array(datasets_dates[-14712:])

df = pd.DataFrame({
    'Time': time,
    'ARIMA': data_baseline['Model_3'],
    'LEAR': data_ml['Predictions'],
    'DARMA': data_darma['Model_2'],
    'Actual': data_baseline['Actual'],
})

df['Time'] = pd.to_datetime(df['Time'])
df = df.set_index('Time')

# If date is 2023-01-01 and next 7 days plot
df = df.loc['2024-06-1':'2024-06-7']

plt.figure(figsize=(8, 4))
plt.plot(df.index, df['Actual'], label='Actual',  color='black', linewidth=1)
plt.plot(df.index, df['ARIMA'], label='ARIMA',  color='blue', linewidth=0.65, linestyle='--')
plt.plot(df.index, df['LEAR'], label='LEAR',  color='red', linewidth=0.65, linestyle='--')
plt.plot(df.index, df['DARMA'], label='DARMA',  color='grey', linewidth=0.65, linestyle='--')

plt.xlabel('Time', fontsize=12)
plt.ylabel('DKK per MWh', fontsize=12)

plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()

plt.show()


ValueError: array length 14712 does not match index length 8736

In [8]:
data_ml['Predictions'].shape

(8736,)

In [13]:
len(datasets_dates)

93576